In [58]:
import json
import os
from typing import List, Tuple, Any
from pathlib import Path
import jsonlines
from allennlp.data.vocabulary import Vocabulary
from allennlp.models.archival import load_archive
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
import torch
from allennlp.common.util import import_module_and_submodules
from allennlp.predictors.predictor import Predictor, JsonDict
import sys
sys.path.append("../")
import logging
from typing import List
# import seaborn as sns
import ipywidgets as widgets
import matplotlib
from IPython.display import display
from box_embeddings.parameterizations.box_tensor import BoxTensor
import tqdm
%matplotlib notebook

In [62]:
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", level=logging.INFO)
logger = logging.getLogger("notebook")

In [60]:
# top level variables
os.chdir('../')
rundir = Path('.allennlp_models/run-20220224_095947-91cs6klj')
model_path = rundir/'model.tar.gz'
config_path = rundir/'config.json'
vocab_file = rundir/'vocabulary'

In [61]:
# load config
with open(config_path) as f:
    config = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '.allennlp_models/run-20220224_095947-91cs6klj/config.json'

In [ ]:
os.getenv('PYTHONPATH')
print('test')

In [ ]:
import_module_and_submodules("box-mlc")

In [ ]:
# Step 1: Load the model
archive = load_archive(
    model_path,
)
# Step 2: Get the predictor
predictor = Predictor.from_archive(
        archive, 
        #args.predictor, 
        dataset_reader_to_load="validation",
        frozen = True
    )
# Switch on the vis mode on model
predictor._model.visualization_mode = True

In [ ]:
# Step 3: Perform predictions
data = list(predictor._dataset_reader.read(config["validation_data_path"]))
logger.info("Done reading data...")
logger.info("Starting to predict")
predictions = []
y_boxes: List[Tuple[List, List]] = []
for x in tqdm.tqdm(data, desc="predicting"):
    r = predictor.predict_instance(x)
    x_box_z = r["x_boxes_z"]
    x_box_Z = r["x_boxes_Z"]
    predictions.append(
        {
            "label_scores": r["scores"],
            "predicted_labels": r["predictions"],
            "true_labels": r["meta"]["labels"],
            "x_box": (x_box_z, x_box_Z),
        }
    )
    if not y_boxes:
        for y_box_z, y_box_Z in zip(r["y_boxes_z"], r["y_boxes_Z"]):
            y_boxes.append((y_box_z, y_box_Z))

In [ ]:
predictions[0]